In [8]:
import pandas as pd
import re
import glob
import os

folder_path = '/Users/fulinq/Documents/SET/MarginModel/MM_History/Merged_MM'
os.chdir(folder_path)

In [20]:
file_pattern = '*_merged_margin_data.xlsx'
date_column_name = 'Effective Date' # Your specific date column
output_filename = 'combined_merged_margin_data.xlsx' # <<< ADJUSTED for Excel output

# Sheet name to read from each Excel file
# IMPORTANT: This should be a string for a single sheet by name
sheet_to_read = 'Margin Data' # <<< CORRECTED from {'Margin Data'}

# --- Processing Steps ---
list_of_dataframes = [] # Initialize here, before the try block

# Construct the full file search pattern
full_pattern = os.path.join(folder_path, file_pattern)

# Find all files matching the pattern
excel_files = glob.glob(full_pattern)

# Read each Excel file and append its DataFrame to the list
for file in excel_files:
    if os.path.basename(file).startswith('~'): # Skip temporary files
        continue
    
    # Corrected sheet_name parameter
    df_single = pd.read_excel(file, sheet_name=sheet_to_read)
    list_of_dataframes.append(df_single)
    print(f"  Successfully read: {os.path.basename(file)}")

# Combine all DataFrames into one
print("\nCombining all DataFrames...")
combined_df = pd.concat(list_of_dataframes, ignore_index=True)
print(f"Combined DataFrame has {len(combined_df)} rows.")

combined_df[date_column_name] = pd.to_datetime(combined_df[date_column_name], errors='coerce')

initial_rows = len(combined_df)
combined_df.dropna(subset=[date_column_name], inplace=True)

combined_df.sort_values(by=date_column_name, inplace=True)

combined_df.tail(5)

  Successfully read: 2022_merged_margin_data.xlsx
  Successfully read: 2025_merged_margin_data.xlsx
  Successfully read: 2021_merged_margin_data.xlsx
  Successfully read: 2019_merged_margin_data.xlsx
  Successfully read: 2023_merged_margin_data.xlsx
  Successfully read: 2020_merged_margin_data.xlsx
  Successfully read: 2024_merged_margin_data.xlsx

Combining all DataFrames...
Combined DataFrame has 26650 rows.


,Product,Market,Type,Maintenance Margin,Effective Date
5631,PSL,Stock Market,Future,580,2025-05-16
5630,PSH,Stock Market,Future,540,2025-05-16
5629,PRM,Stock Market,Future,640,2025-05-16
5627,OSP,Stock Market,Future,1300,2025-05-16
5546,EUR/USD,Currency Market,Future,760,2025-05-16


In [13]:
df_p = combined_df.copy()
df_p.head()

,Product,Market,Type,Maintenance Margin,Effective Date
0,BANK,Index Market,Future,14900,28 January 2022
1,COMM,Index Market,Future,9320,28 January 2022
2,ENERG,Index Market,Future,5620,28 January 2022
3,FOOD,Index Market,Future,2560,28 January 2022
4,ICT,Index Market,Future,5620,28 January 2022


In [14]:
df_p = df_p[['Product', 'Effective Date', 'Maintenance Margin']]
df_p

,Product,Effective Date,Maintenance Margin
0,BANK,28 January 2022,14900
1,COMM,28 January 2022,9320
2,ENERG,28 January 2022,5620
3,FOOD,28 January 2022,2560
4,ICT,28 January 2022,5620
...,...,...,...
26645,UNIQ,26 January 2024,160
26646,VGI,26 January 2024,140
26647,VNG,26 January 2024,120
26648,WHA,26 January 2024,200


In [15]:
df_p['Effective Date'] = pd.to_datetime(df_p['Effective Date'], errors='coerce')
df_p['Maintenance Margin'] = pd.to_numeric(df_p['Maintenance Margin'], errors='coerce')
df_p.head()

,Product,Effective Date,Maintenance Margin
0,BANK,2022-01-28,14900
1,COMM,2022-01-28,9320
2,ENERG,2022-01-28,5620
3,FOOD,2022-01-28,2560
4,ICT,2022-01-28,5620


In [16]:
pivoted_df = df_p.pivot_table(index='Effective Date', columns='Product', values='Maintenance Margin', aggfunc='first')
pivoted_df.reset_index(inplace=True)
pivoted_df

Product,Effective Date,10 Baht Gold,3M BIBOR,50 Baht Gold,5Y Government Bond,AAV,ADVANC,AEONTS,AMATA,AOT,...,TVO,True,UNIQ,USD,USD/JPY,VGI,VNG,VNT,WHA,WHAUP
0,2019-04-11,2400.0,380.0,12000.0,4500.0,220.0,7060.0,NaN,1200.0,2200.0,...,1360.0,280.0,560.0,260.0,NaN,380.0,360.0,NaN,180.0,NaN
1,2019-04-25,2400.0,380.0,12000.0,4500.0,220.0,7060.0,NaN,1200.0,2200.0,...,1360.0,280.0,560.0,260.0,NaN,380.0,360.0,NaN,180.0,NaN
2,2019-04-26,2400.0,380.0,12000.0,4500.0,220.0,7060.0,NaN,1200.0,2200.0,...,1360.0,280.0,560.0,260.0,NaN,380.0,360.0,NaN,180.0,NaN
3,2019-05-03,2400.0,380.0,12000.0,4500.0,220.0,7060.0,NaN,1200.0,2200.0,...,1360.0,280.0,560.0,260.0,NaN,380.0,360.0,NaN,180.0,NaN
4,2019-05-17,2600.0,380.0,13000.0,2800.0,180.0,5400.0,NaN,1200.0,2200.0,...,1200.0,260.0,560.0,260.0,NaN,460.0,360.0,NaN,180.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2025-04-22,11560.0,1920.0,57800.0,3940.0,160.0,11460.0,6500.0,2720.0,3240.0,...,1440.0,720.0,100.0,520.0,820.0,380.0,100.0,NaN,420.0,280.0
183,2025-04-28,11560.0,1920.0,57800.0,3940.0,160.0,11460.0,6500.0,2720.0,3240.0,...,1440.0,720.0,160.0,520.0,820.0,380.0,100.0,NaN,420.0,440.0
184,2025-04-30,13580.0,1920.0,67900.0,3940.0,160.0,11460.0,6500.0,2720.0,3240.0,...,1440.0,720.0,160.0,520.0,820.0,380.0,100.0,NaN,420.0,440.0
185,2025-05-08,13580.0,1920.0,67900.0,3940.0,160.0,11460.0,6500.0,2720.0,3240.0,...,1440.0,720.0,160.0,520.0,820.0,380.0,100.0,NaN,420.0,440.0


In [17]:
with pd.ExcelWriter('combined_merged_margin_data.xlsx') as writer:
    pivoted_df.to_excel(writer, index=False, sheet_name='Maintenance Margin')